In [ ]:
%%HTML
<style>
    body {
        --vscode-font-family: "Noto Serif"
    }
</style>

# Monotonic Stack & Queue Patterns

## 1. Concept

A monotonic stack keeps elements (usually indices) so their corresponding values are strictly increasing or decreasing.
Each element is pushed and popped at most once → O(n) total.
You pop when the incoming element violates the monotonic invariant, resolving queries for popped items.

Why indices? They let you compute distances (spans, widths) and re-access values. Each time element pop from stack, it will make a contribution to the final result:

What Contribution Means: Each element in the array can "contribute" to the answer over a certain range defined by:
- The next greater/smaller element to the left
- The next greater/smaller element to the right
- The stack helps identify these bounds efficiently.

## 2. Classic Problems Overview
- Next Greater Element (NGE)
- Daily Temperatures (span until warmer)
- Stock Span (backward span of <= prices)
- Largest Rectangle in Histogram (width via nearest smaller left/right)
- Trapping Rain Water (stack version)
- Sum of Subarray Minimums (contribution via nearest smaller constraints)
- Remove K Digits (greedy monotonic increasing digits)
- Sliding Window Maximum (monotonic queue / deque)
- Asteroid Collision (directional resolution)

### Next Greater Element (NGE)
**Problem Statement:** Given an integer array, for each element find the next element to its right that is strictly greater; if none, return -1.

**Intuition:** Maintain a decreasing stack of indices. When a new element is greater than stack top, it resolves that index's answer. Each index is pushed/popped at most once.

**Why it Works:** 
- The stack always stores indices of elements whose next greater element we have not yet discovered, in strictly decreasing value order (top is the smallest among them). 
- The first time we see a value greater than nums[j], it must be the closest greater to the right (because every element between j and current index i was processed and was <= nums[j], otherwise j would have been popped earlier). After popping j we never revisit it. 
- Each index is pushed exactly once and popped at most once → linear time.

**Complexity:** O(n) time, O(n) space (answers + stack).

In [ ]:
package main

import "fmt"

// Next Greater Element (NGE)
func nextGreaterElement(nums []int) []int {
    n := len(nums)
    result := make([]int, n)
    for i := range result {
        result[i] = -1
    }
    var stack []int // holds indices with decreasing values
    
    for i, val := range nums {
        fmt.Printf("i=%d, val=%d, stack=%v, result=%v\n", i, val, stack, result)
        for len(stack) > 0 && nums[stack[len(stack)-1]] < val {
            topIdx := stack[len(stack)-1]
            stack = stack[:len(stack)-1]
            result[topIdx] = val
            fmt.Printf("    i=%d, val=%d, stack=%v, result=%v\n", i, val, stack, result)
        }
        stack = append(stack, i)
        fmt.Printf("i=%d, val=%d, stack=%v, result=%v\n\n", i, val, stack, result)
    }
    return result
}

%%
result := nextGreaterElement([]int{3, 2, 1, 4, 3})
fmt.Printf("Final result: %v\n", result)

### Daily Temperatures
**Problem Statement:** Given daily temperatures, return for each day how many days until a warmer temperature; 0 if none.

**Intuition:** Use a decreasing stack of indices (temperatures strictly descending). When a warmer temperature appears, pop indices and compute distance.

**Why it Works:** For an index j, the first time we encounter a temperature higher than temps[j] at index i, all intervening temperatures were <= temps[j]; otherwise j would have been resolved earlier. Thus i is the closest warmer day. Strict decrease ensures each index is stored only until its answer is known. Push/pop once → O(n).

**Complexity:** O(n) time, O(n) space.

In [ ]:
// Daily Temperatures
func dailyTemperatures(temps []int) []int {
    n := len(temps)
    answer := make([]int, n)
    var stack []int // indices with strictly decreasing temperatures
    
    for i, t := range temps {
        for len(stack) > 0 && temps[stack[len(stack)-1]] < t {
            j := stack[len(stack)-1]
            stack = stack[:len(stack)-1]
            answer[j] = i - j
        }
        stack = append(stack, i)
    }
    return answer
}

%%
result := dailyTemperatures([]int{73, 74, 75, 71, 69, 72, 76, 73})
fmt.Printf("Daily temperatures result: %v\n", result)

### Stock Span
**Problem Statement:** For each day's price, compute the number of consecutive prior days (including today) with price <= today's price.

**Intuition:** Maintain a stack of (price, span) in decreasing price order. Aggregate spans while popping smaller or equal prices.

**Why it Works:** Any popped element had price <= current price, so its entire accumulated span is merged. Because spans collapse into one record, each day is popped at most once. Prices remaining on stack are strictly greater and thus bound the span window on the left. This guarantees linear total operations.

**Complexity:** O(n) time, O(n) space worst-case.

In [ ]:
// Stock Span
type PriceSpan struct {
    Price int
    Span  int
}

func stockSpan(prices []int) []int {
    var spans []int
    var stack []PriceSpan // (price, accumulated_span)
    
    for _, price := range prices {
        span := 1
        fmt.Printf("price=%d, span=%d, stack=%v, spans=%v\n", price, span, stack, spans)
        
        for len(stack) > 0 && stack[len(stack)-1].Price <= price {
            span += stack[len(stack)-1].Span
            stack = stack[:len(stack)-1]
            fmt.Printf("  price=%d, span=%d, stack=%v, spans=%v\n", price, span, stack, spans)
        }
        
        stack = append(stack, PriceSpan{price, span})
        spans = append(spans, span)
        fmt.Printf("price=%d, span=%d, stack=%v, spans=%v\n\n", price, span, stack, spans)
    }
    return spans
}

%%
result := stockSpan([]int{100, 80, 60, 55, 50, 45, 95})
fmt.Printf("Stock span result: %v\n", result)

### Largest Rectangle in Histogram
**Problem Statement:** Given bar heights (array of non-negative integers), find the area of the largest rectangle fully contained within the histogram.

**Intuition:** 
- For each bar as the limiting (minimum) height, its maximal width extends to just before the previous smaller bar on the left and just before the next smaller bar on the right. 
- A monotonic increasing stack (by height) lets us know when a bar's right boundary is found (current height < stack top height). Pop and compute area on the fly.

**Why it Works:** 
- Each bar is pushed once. 
- When a shorter bar arrives, every taller bar above it has now discovered its next smaller element (right boundary). 
- The height popped has left boundary = new stack top (or -1 if empty). 
- Right boundary = current index (exclusive). This yields width = right_index - left_index - 1. 
- The invariant (increasing heights) ensures correctness and linear total pops.

**Complexity:** O(n) time, O(n) space for stack.

In [ ]:
// Largest Rectangle in Histogram
func largestRectangleArea(heights []int) int {
    extended := append(heights, 0) // Append sentinel zero to flush stack
    var stack []int // indices of increasing heights
    best := 0
    
    for i, h := range extended {
        fmt.Printf("i=%d, h=%d, stack=%v, best=%d\n", i, h, stack, best)
        for len(stack) > 0 && extended[stack[len(stack)-1]] > h {
            mid := stack[len(stack)-1]
            stack = stack[:len(stack)-1]
            leftIndex := -1
            if len(stack) > 0 {
                leftIndex = stack[len(stack)-1]
            }
            width := i - leftIndex - 1
            area := extended[mid] * width
            if area > best {
                best = area
            }
            fmt.Printf("    mid=%d, leftIndex=%d, width=%d, area=%d, best=%d, stack=%v\n", 
                       mid, leftIndex, width, area, best, stack)
        }
        stack = append(stack, i)
        fmt.Printf("i=%d, h=%d, stack=%v, best=%d\n\n", i, h, stack, best)
    }
    return best
}

%%
result := largestRectangleArea([]int{1, 2, 3, 4, 5, 3, 1, 7})
fmt.Printf("Largest rectangle area: %d\n", result)

### Trapping Rain Water (Stack Version)
**Problem Statement:** Given non-negative integer heights representing elevation map bars (width 1), compute how much water can be trapped after raining.

**Intuition:** Use a stack of indices of non-decreasing heights. When a higher bar appears, it may form a container with a previous higher (left) boundary. Pop the middle bar as the "bottom" and compute water = width * (min(left_height, right_height) - bottom_height).

**Why it Works:** Water trapped above a bar depends on the nearest higher (or equal) bars on both sides. The monotonic increase ensures that when a taller right bar arrives, all lower bars between it and a taller (or equal) left boundary can have their trapped water resolved exactly once. Each index is pushed/popped at most once.

**Complexity:** O(n) time, O(n) space.

In [ ]:
// Trapping Rain Water (Stack Version)
func trapRainWater(height []int) int {
    var stack []int
    water := 0
    
    for i, h := range height {
        fmt.Printf("i=%d, h=%d, stack=%v, water=%d\n", i, h, stack, water)
        for len(stack) > 0 && height[stack[len(stack)-1]] < h {
            mid := stack[len(stack)-1]
            stack = stack[:len(stack)-1]
            fmt.Printf("   i=%d, h=%d, stack=%v, water=%d <- stack popped\n", i, h, stack, water)
            if len(stack) == 0 {
                break
            }
            left := stack[len(stack)-1]
            width := i - left - 1
            boundedHeight := min(height[left], h) - height[mid]
            fmt.Printf("   left=%d, width=%d, boundHeight=%d\n", left, width, boundedHeight)
            if boundedHeight > 0 {
                water += width * boundedHeight
            }
            fmt.Printf("   i=%d, h=%d, stack=%v, water=%d\n", i, h, stack, water)
        }
        stack = append(stack, i)
        fmt.Printf("   i=%d, h=%d, stack=%v, water=%d\n\n", i, h, stack, water)
    }
    return water
}

func min(a, b int) int {
    if a < b {
        return a
    }
    return b
}

%%
result := trapRainWater([]int{0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1})
fmt.Printf("Trapped rain water: %d\n", result)

### Sum of Subarray Minimums
**Problem Statement:** Given an array, compute the sum of the minimum of every contiguous subarray (mod `1e9+7` if needed).

**Intuition:** Each element contributes its value times the number of subarrays in which it is the minimum. Need counts of subarrays where it is the unique chosen minimum: distances to previous strictly smaller and next smaller-or-equal elements. Use two monotonic passes.

**Why it Works:** The contribution technique decomposes the global sum into disjoint sets of subarrays each charged to exactly one minimum element, achieved by consistent tie-breaking (strict on left, non-strict on right). Monotonic stacks find nearest smaller boundaries in linear time.

**Complexity:** O(n) time, O(n) space.

In [ ]:
// Sum of Subarray Minimums
func sumSubarrayMins(arr []int) int {
    const MOD = 1000000007
    n := len(arr)
    prevSmaller := make([]int, n)
    var stack []int
    
    // Previous strictly smaller
    for i, x := range arr {
        for len(stack) > 0 && arr[stack[len(stack)-1]] >= x {
            stack = stack[:len(stack)-1]
        }
        if len(stack) == 0 {
            prevSmaller[i] = -1
        } else {
            prevSmaller[i] = stack[len(stack)-1]
        }
        stack = append(stack, i)
    }
    
    nextSmallerEqual := make([]int, n)
    stack = stack[:0] // clear stack
    
    // Next smaller or equal
    for i := n - 1; i >= 0; i-- {
        x := arr[i]
        for len(stack) > 0 && arr[stack[len(stack)-1]] > x {
            stack = stack[:len(stack)-1]
        }
        if len(stack) == 0 {
            nextSmallerEqual[i] = n
        } else {
            nextSmallerEqual[i] = stack[len(stack)-1]
        }
        stack = append(stack, i)
    }
    
    total := int64(0)
    for i, x := range arr {
        left := i - prevSmaller[i]
        right := nextSmallerEqual[i] - i
        total = (total + int64(x)*int64(left)*int64(right)) % MOD
    }
    return int(total)
}

%%
result := sumSubarrayMins([]int{3, 1, 2, 4})
fmt.Printf("Sum of subarray minimums: %d\n", result)

### Remove K Digits
**Problem Statement:** Given a numeric string and integer k, remove k digits so the resulting number is the smallest possible (no leading zeros unless result is 0).

**Intuition:** Greedy monotonic increasing stack of digits: while current digit is smaller than the stack top and we still can remove digits (k>0), pop to reduce future magnitude.

**Why it Works:** A larger digit earlier increases the lexicographic / numeric value more than later positions. Removing a left-side larger digit when a smaller digit appears yields globally smaller number. Stack ensures minimal local prefix; remaining k leads to trimming the largest trailing digits. Leading zeros are stripped.

**Complexity:** O(n) time, O(n) space.

In [ ]:
import "strings"

// Remove K Digits
func removeKdigits(num string, k int) string {
    var stack []rune
    
    for _, ch := range num {
        for k > 0 && len(stack) > 0 && stack[len(stack)-1] > ch {
            stack = stack[:len(stack)-1]
            k--
        }
        stack = append(stack, ch)
    }
    
    if k > 0 {
        // remove from end if still left
        stack = stack[:len(stack)-k]
    }
    
    result := string(stack)
    result = strings.TrimLeft(result, "0")
    if result == "" {
        return "0"
    }
    return result
}

%%
result := removeKdigits("1234567", 3)
fmt.Printf("Remove K digits result: %s\n", result)

### Sliding Window Maximum (Monotonic Queue)
**Problem Statement:** Given an array and window size k, return the max in each sliding window of size k.

**Intuition:** Maintain a deque of indices with decreasing values (front is current max). Drop indices that fall out of the window and pop from back while new value >= back's value.

**Why it Works:** Deque holds only candidates that can still become maximum for future windows in strictly decreasing order. Removing dominated (smaller) elements early guarantees each index enters and leaves deque once → O(n).

**Complexity:** O(n) time, O(k) space.

In [ ]:
// Sliding Window Maximum (Monotonic Queue)
func slidingWindowMax(nums []int, k int) []int {
    if k <= 0 {
        return []int{}
    }
    var dq []int // indices, values strictly decreasing
    var out []int
    
    for i, val := range nums {
        for len(dq) > 0 && nums[dq[len(dq)-1]] <= val {
            dq = dq[:len(dq)-1]
        }
        dq = append(dq, i)
        
        // Remove if out of the window (on the left)
        if len(dq) > 0 && dq[0] <= i-k {
            dq = dq[1:]
        }
        
        // Start appending only when window gets its size (no window for the first k-1 index)
        if i >= k-1 {
            out = append(out, nums[dq[0]])
        }
    }
    return out
}

%%
result := slidingWindowMax([]int{1, 3, -1, -3, 5, 3, 6, 7}, 3)
fmt.Printf("Sliding window maximum: %v\n", result)

### Asteroid Collision
**Problem Statement:** Given a list of integers representing asteroids (sign = direction, magnitude = size). Positive moves right, negative moves left. When two meet, smaller explodes; if equal both explode. Return state after all collisions.

**Intuition:** Stack holds asteroids moving right. When a new left-moving asteroid arrives (<0), resolve collisions while the top is a smaller right mover. Continue until it either explodes, all smaller right movers are removed, or encounters an equal-size right mover.

**Why it Works:** Only possible collisions are between a right-moving asteroid already seen and a new left-moving one. Future asteroids cannot affect resolved past collisions. Each asteroid is pushed once and popped at most once. Directional constraint reduces pairwise checks to amortized constant per asteroid.

**Complexity:** O(n) time, O(n) space.

In [ ]:
// Asteroid Collision
func asteroidCollision(asteroids []int) []int {
    var stack []int
    
    for _, a := range asteroids {
        alive := true
        for alive && a < 0 && len(stack) > 0 && stack[len(stack)-1] > 0 {
            top := stack[len(stack)-1]
            if top < -a { // stack top explodes
                stack = stack[:len(stack)-1]
                continue
            }
            if top == -a { // both explode
                stack = stack[:len(stack)-1]
            }
            alive = false // current explodes or both destroyed
        }
        if alive {
            stack = append(stack, a)
        }
    }
    return stack
}

%%
result := asteroidCollision([]int{5, 10, -5})
fmt.Printf("Asteroid collision result: %v\n", result)